In [1]:
import os

In [2]:
# to check current working directory
%pwd

'd:\\Ineuron_projects\\Chest-Disease-Classification-from-Chest-CT-scan-images\\research'

In [3]:
#
os.chdir("../")

In [4]:
%pwd

'd:\\Ineuron_projects\\Chest-Disease-Classification-from-Chest-CT-scan-images'

In [6]:
# for entity 
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
'''
This code defines a dataclass named `DataIngestionConfig` using the `dataclasses` library and `Path` objects from the `pathlib` library in Python. Here's a breakdown:

1. **Imports:**
   - `from dataclasses import dataclass`: This line imports the `dataclass` decorator from the `dataclasses` library. This decorator helps create classes with automatic functionalities like `__init__` and getters.
   - `from pathlib import Path`: This imports the `Path` class from the `pathlib` library. This class offers a convenient way to handle file paths in Python.

2. **Dataclass Definition:**
   - `@dataclass(frozen=True)`: This line defines a dataclass named `DataIngestionConfig`. The `@dataclass` decorator tells Python to generate boilerplate code for the class, including an `__init__` method and methods to access attributes. The `frozen=True` argument makes the class immutable, meaning its attributes cannot be changed after creation.

3. **Attributes:**
   - `root_dir: Path`: This defines an attribute named `root_dir` with a type hint of `Path`. This indicates that the `root_dir` attribute should hold a path object representing the root directory where data will be stored.
   - `source_URL: str`: This defines an attribute named `source_URL` with a type hint of `str`. This attribute stores the URL from which the data will be downloaded.
   - `local_data_file: Path`: This defines an attribute named `local_data_file` with a type hint of `Path`. This attribute represents the path to the local file where the downloaded data will be saved.
   - `unzip_dir: Path`: This defines an attribute named `unzip_dir` with a type hint of `Path`. This attribute represents the path to the directory where the downloaded data will be unzipped (if applicable).

In essence, this code defines a configuration class specifically for data ingestion tasks. It ensures data paths are handled correctly (using `Path`) and makes the configuration object immutable (`frozen=True`). This promotes data integrity and clarity in your code. '''


"\nThis code defines a dataclass named `DataIngestionConfig` using the `dataclasses` library and `Path` objects from the `pathlib` library in Python. Here's a breakdown:\n\n1. **Imports:**\n   - `from dataclasses import dataclass`: This line imports the `dataclass` decorator from the `dataclasses` library. This decorator helps create classes with automatic functionalities like `__init__` and getters.\n   - `from pathlib import Path`: This imports the `Path` class from the `pathlib` library. This class offers a convenient way to handle file paths in Python.\n\n2. **Dataclass Definition:**\n   - `@dataclass(frozen=True)`: This line defines a dataclass named `DataIngestionConfig`. The `@dataclass` decorator tells Python to generate boilerplate code for the class, including an `__init__` method and methods to access attributes. The `frozen=True` argument makes the class immutable, meaning its attributes cannot be changed after creation.\n\n3. **Attributes:**\n   - `root_dir: Path`: This de

In [17]:
# testing the paths 

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
print(CONFIG_FILE_PATH)
print("*" * 10)
print(PARAMS_FILE_PATH)

config\config.yaml
**********
params.yaml


In [12]:
# Update the configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [18]:
# Update the components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
# pipeline 

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-13 16:03:26,280: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-13 16:03:26,283: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-13 16:03:26,285: INFO: common: created directory at: artifacts]
[2024-03-13 16:03:26,287: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-13 16:03:26,288: INFO: 3654218236: Downloading data from https://drive.google.com/file/d/13bzvXuPl51k5btdaI1KAiYE4HjIrbB2Q/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=13bzvXuPl51k5btdaI1KAiYE4HjIrbB2Q
From (redirected): https://drive.usercontent.google.com/download?id=13bzvXuPl51k5btdaI1KAiYE4HjIrbB2Q&confirm=t&uuid=b22c7fef-3933-4b89-b827-2d19ec7f367b
To: d:\Ineuron_projects\Chest-Disease-Classification-from-Chest-CT-scan-images\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:13<00:00, 3.66MB/s]

[2024-03-13 16:03:41,329: INFO: 3654218236: Downloaded data from https://drive.google.com/file/d/13bzvXuPl51k5btdaI1KAiYE4HjIrbB2Q/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
